# Dataset Cleaning 

The main objectives of this notebook are: 
- Import and merge together multiple datasets into a features df 
- Get familiar with the features df 
- Determine if there are data quality issues 
- Resolve data quality issues 

In [105]:
import pandas as pd 
from datetime import datetime, timedelta
import sys
import os 

# Manually add path to read from another folder 
path2add = os.path.normpath(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), os.path.pardir, "utils"))
)
if not (path2add in sys.path):
    sys.path.append(path2add)

from cleaning import *

In [106]:
# Setting the start and end date 
end_date = datetime.today()
start_date = end_date - timedelta(days=500*365)
print(start_date)

1525-01-26 11:44:04.635291


### Data Importing 

In [107]:
oil_df = get_oil_data(oil_api_keys)
oil_df[:2]

[            libya_oil_production
 2000-01-01          1.469428e+06
 2001-01-01          1.427842e+06
 2002-01-01          1.383184e+06
 2003-01-01          1.805719e+06
 2004-01-01          1.582475e+06
 2005-01-01          1.721293e+06
 2006-01-01          1.809577e+06
 2007-01-01          1.844703e+06
 2008-01-01          1.781000e+06
 2009-01-01          1.623000e+06
 2010-01-01          1.688000e+06
 2011-01-01          4.600000e+05
 2012-01-01          1.390000e+06
 2013-01-01          9.000000e+05
 2014-01-01          4.602740e+05
 2015-01-01          4.000000e+05
 2016-01-01          3.901644e+05
 2017-01-01          8.100000e+05
 2018-01-01          9.673633e+05
 2019-01-01          1.100000e+06
 2020-01-01          3.000000e+05
 2021-01-01          1.206667e+06
 2022-01-01          1.001250e+06
 2023-01-01          1.179167e+06
 2024-01-01          1.300000e+06
 2025-01-01          1.400000e+06,
             kazak_oil_production
 2000-01-01          7.270479e+05
 2001-01-01  

In [108]:
usa_oil_production = (
    pd.read_csv('../data/usa_crude_oil_production.csv')
    .sort_values(['Month'], ascending=True)
    .rename(columns={'U.S. Field Production of Crude Oil Thousand Barrels per Day': 'usa_oil_production'})
)

In [109]:
usa_oil_production['Month'] = pd.to_datetime(usa_oil_production['Month'], format='%b %Y').dt.strftime('%Y-%m-%d')
usa_oil_production.set_index('Month', inplace=True)
usa_oil_production.index = usa_oil_production.index.astype('datetime64[ns]')
usa_oil_production.rename_axis('Date', inplace=True)

usa_oil_production['usa_oil_production'] *= 1000

usa_oil_production

,usa_oil_production
Date,
1920-04-01,1165000
1921-04-01,1341000
1922-04-01,1506000
1923-04-01,1967000
1924-04-01,1994000
...,...
2019-09-01,12584000
2020-09-01,10926000
2021-09-01,10921000


In [110]:
financial_df = get_financial_data(financial_api_keys,start_date,end_date)
financial_df[:2]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[            crude_price
 Date                   
 2000-08-23    32.049999
 2000-08-24    31.629999
 2000-08-25    32.049999
 2000-08-28    32.869999
 2000-08-29    32.720001
 ...                 ...
 2024-09-19    71.949997
 2024-09-20    71.919998
 2024-09-23    70.370003
 2024-09-24    71.559998
 2024-09-25    69.690002
 
 [6048 rows x 1 columns],
                  sp_500
 Date                   
 2000-09-18  1467.500000
 2000-09-19  1478.500000
 2000-09-20  1469.500000
 2000-09-21  1469.500000
 2000-09-22  1468.500000
 ...                 ...
 2024-09-19  5717.750000
 2024-09-20  5699.990234
 2024-09-23  5776.750000
 2024-09-24  5792.000000
 2024-09-25  5779.000000
 
 [6067 rows x 1 columns]]

In [111]:
dataframes = axis_manipulation(oil_df)

In [112]:
# Importing oil consumption data 
world_oil_consumption = pd.read_csv("../data/world_oil_consumption.csv")
world_oil_consumption.rename(columns={"Oil consumption - TWh":"world_oil_consumption"},inplace=True)
world_oil_consumption.drop(world_oil_consumption[world_oil_consumption['Entity'] != "World"].index, inplace=True)
world_oil_consumption.drop(columns={'Entity','Code'},inplace=True)

In [113]:
world_oil_consumption['Year'] = pd.to_datetime(world_oil_consumption['Year'], format='%Y').dt.strftime('%Y-%m-%d')
world_oil_consumption.set_index('Year', inplace=True)
world_oil_consumption.rename_axis(index="Date",inplace=True) 
world_oil_consumption.index = pd.to_datetime(world_oil_consumption.index)

In [114]:
world_oil_consumption=world_oil_consumption.resample('D').ffill()
usa_oil_production=usa_oil_production.resample('D').ffill()


In [115]:
oil_df = resample_data(oil_df)
oil_df

[            libya_oil_production
 Date                            
 2000-01-01           1469427.589
 2000-01-02           1469427.589
 2000-01-03           1469427.589
 2000-01-04           1469427.589
 2000-01-05           1469427.589
 ...                          ...
 2024-12-28           1300000.000
 2024-12-29           1300000.000
 2024-12-30           1300000.000
 2024-12-31           1300000.000
 2025-01-01           1400000.000
 
 [9133 rows x 1 columns],
             kazak_oil_production
 Date                            
 2000-01-01          7.270479e+05
 2000-01-02          7.270479e+05
 2000-01-03          7.270479e+05
 2000-01-04          7.270479e+05
 2000-01-05          7.270479e+05
 ...                          ...
 2024-12-28          1.855479e+06
 2024-12-29          1.855479e+06
 2024-12-30          1.855479e+06
 2024-12-31          1.855479e+06
 2025-01-01          2.116438e+06
 
 [9133 rows x 1 columns],
             qatar_oil_production
 Date                     

### Merging datasets 

In [116]:
oil_df = merge_df(oil_df)
oil_df = oil_df.loc[:'2024-09-20']
oil_df

,libya_oil_production,kazak_oil_production,qatar_oil_production,iran_oil_production,kuwait_oil_production,uae_oil_production,saudi_oil_production,iraq_oil_production
Date,,,,,,,,
2000-01-01,1469427.589,7.270479e+05,688493.1507,3.661000e+06,NaN,2190000.0,8.090000e+06,2525000.0
2000-01-02,1469427.589,7.270479e+05,688493.1507,3.661000e+06,NaN,2190000.0,8.090000e+06,2525000.0
2000-01-03,1469427.589,7.270479e+05,688493.1507,3.661000e+06,NaN,2190000.0,8.090000e+06,2525000.0
2000-01-04,1469427.589,7.270479e+05,688493.1507,3.661000e+06,NaN,2190000.0,8.090000e+06,2525000.0
2000-01-05,1469427.589,7.270479e+05,688493.1507,3.661000e+06,NaN,2190000.0,8.090000e+06,2525000.0
...,...,...,...,...,...,...,...,...
2024-09-16,1300000.000,1.855479e+06,580000.0000,3.194986e+06,2480500.0,2994750.0,9.266582e+06,4110000.0
2024-09-17,1300000.000,1.855479e+06,580000.0000,3.194986e+06,2480500.0,2994750.0,9.266582e+06,4110000.0
2024-09-18,1300000.000,1.855479e+06,580000.0000,3.194986e+06,2480500.0,2994750.0,9.266582e+06,4110000.0


In [117]:
financial_df = merge_df(financial_df)
financial_df

,crude_price,sp_500,vix,usd
Date,,,,
1990-01-02,NaN,NaN,17.240000,NaN
1990-01-03,NaN,NaN,18.190001,NaN
1990-01-04,NaN,NaN,19.219999,NaN
1990-01-05,NaN,NaN,20.110001,NaN
1990-01-08,NaN,NaN,20.260000,NaN
...,...,...,...,...
2024-09-20,71.919998,5699.990234,16.150000,100.422997
2024-09-23,70.370003,5776.750000,15.890000,100.559998
2024-09-24,71.559998,5792.000000,15.390000,100.158997


In [118]:
features_df = [financial_df,oil_df,usa_oil_production,world_oil_consumption]

In [119]:
features_df = merge_df(features_df)
features_df

,crude_price,sp_500,vix,usd,libya_oil_production,kazak_oil_production,qatar_oil_production,iran_oil_production,kuwait_oil_production,uae_oil_production,saudi_oil_production,iraq_oil_production,usa_oil_production,world_oil_consumption
Date,,,,,,,,,,,,,,
1920-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1097000.0,NaN
1920-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1097000.0,NaN
1920-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1097000.0,NaN
1920-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1097000.0,NaN
1920-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1097000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-20,71.919998,5699.990234,16.15,100.422997,1300000.0,1.855479e+06,580000.0,3.194986e+06,2480500.0,2994750.0,9.266582e+06,4110000.0,NaN,NaN
2024-09-23,70.370003,5776.750000,15.89,100.559998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-09-24,71.559998,5792.000000,15.39,100.158997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Missing Data 

In [120]:
features_df.isna().sum()

crude_price              32206
sp_500                   32187
vix                      29503
usd                      32127
libya_oil_production     29224
kazak_oil_production     29224
qatar_oil_production     29224
iran_oil_production      29224
kuwait_oil_production    29955
uae_oil_production       29224
saudi_oil_production     29224
iraq_oil_production      29224
usa_oil_production         146
world_oil_consumption    17069
dtype: int64

##### Observations
- There are large amounts of missing data due to VIX data being available from 1990 while other data begins much later. 
- Two techniques will be used to fill in missing data. 
    - If there is any missing data after the first recorded occurence, it will be filled with a rolling 7 day average. 
      Which will increase the dataset size while keeping data integrity high. 
    - All remaining NaN values that can't be filled with their averages will be dropped. 

##### Impact 
- Utilizing the preceeding techniques will increase the data quality.

### Data Pre-processing Pipeline

A data cleaning and processing pipeline will be implemented utilizing the techniques described above. 
- Fill NaN values with rolling 7 day average 
- Drop remaining NaN values 
- Output data as csv file 

In [121]:
def cleaning_pipeline(data: pd.DataFrame, output_path: str):
    """
        
    Args:
        df : pd.DataFrames 

    Returns:
        pd.DataFrames: dataframe 
    """
    data = data.fillna(data.rolling(7, min_periods=1).mean())
    data = data.dropna()
    data.to_csv(output_path)

In [122]:
output = "../data/cleaned_dataset.csv"
cleaning_pipeline(features_df, output)